In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(
    nn.Linear(20, 256), 
    nn.ReLU(), 
    nn.Linear(256, 10)
    )

X = torch.rand(2, 20)
net(X)

tensor([[ 0.0712, -0.0503,  0.1727, -0.1083,  0.0964,  0.0122, -0.1126,  0.0207,
          0.0701,  0.2101],
        [ 0.1148, -0.0506,  0.1535, -0.1318,  0.1187, -0.1856,  0.1152,  0.1064,
          0.0138,  0.1401]], grad_fn=<AddmmBackward0>)

### 自定义块

In [2]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [9]:
net = MLP()
net(X)

tensor([[-0.1666, -0.1100,  0.0935,  0.0222, -0.0625, -0.1052,  0.1436, -0.0142,
          0.0040, -0.1067],
        [-0.2709, -0.0257,  0.0015,  0.0686,  0.0193, -0.1634,  0.1783, -0.0643,
         -0.1025, -0.0940]], grad_fn=<AddmmBackward0>)

In [10]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self.add_module(str(idx), module)

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

In [11]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
    
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X= self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()
    
net = FixedHiddenMLP()
net(X)

tensor(-0.1228, grad_fn=<SumBackward0>)

In [12]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(20, 64), 
            nn.ReLU(), 
            nn.Linear(64, 32), 
            nn.ReLU()
            )
        self.linear = nn.Linear(32, 16)
    
    def forward(self, X):
        return self.linear(self.net(X))
    
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)